<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Tesi-MLOPs-pipeline-industriale-Light/blob/main/Script_For_Confusion_matrix_and_R2__in_FromMasks_Training_Convergence_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L'analisi della coerenza del Ground Truth ha evidenziato un errore sistematico di labeling in 49 campioni su 107. Grazie a un algoritmo di verifica basato sulla topologia del foro (analisi del centroide), è stato possibile correggere automaticamente il dataset di validazione. Questo passaggio non solo ha permesso una valutazione corretta del modello UNet++, ma dimostra la robustezza della pipeline ROIA anche a fronte di dati di supervisione umani rumorosi."

Si è proceduto alla pulizia e successivamente al calcolo delle metriche di interesse Recall, confusione, e R2

In [ ]:
!pip install -q seaborn matplotlib scikit-learn


CHECK CHE CAZZZO STIAMO FACENDO


In [ ]:
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm

# --- SET PERCORSI ---
GT_DIR = Path("/content/drive/MyDrive/GOLDEN_GT_FIXED/masks")
PRED_DIR = Path("/content/drive/MyDrive/UNETPPMaschereInferenza")
RADIO_DIR = Path("/content/drive/MyDrive/GOLDEN_DATASET_CANONICO_RADIOGRAFICO/Full_Images")

# --- PARAMETRI ---
n_campioni = 5  # Quanti fori vuoi vedere?

def visual_audit():
    gt_files = {f.stem.lower(): f for f in GT_DIR.glob("*.png")}
    pred_files = {f.stem.lower(): f for f in PRED_DIR.glob("*.png")}

    # Cerchiamo l'intersezione reale
    stems = sorted(list(set(gt_files.keys()) & set(pred_files.keys())))

    if not stems:
        print("❌ Nessun match tra GT e PRED. Controlla i nomi dei file!")
        return

    print(f"🧐 Analisi visiva su {n_campioni} campioni casuali...")

    fig, axes = plt.subplots(n_campioni, 3, figsize=(15, 5*n_campioni))

    for i, stem in enumerate(stems[:n_campioni]):
        # Carica maschere
        m_gt = cv2.imread(str(gt_files[stem]), 0)
        m_pred = cv2.imread(str(pred_files[stem]), 0)

        # Cerca la radio (nome immagine potrebbe essere leggermente diverso)
        radio_path = list(RADIO_DIR.glob(f"*{stem}*"))
        if radio_path:
            img = cv2.imread(str(radio_path[0]), 0)
            img = cv2.resize(img, (512, 512))
        else:
            img = np.zeros((512,512)) # Se manca la radio facciamo sfondo nero

        # Overlay Danno (Giallo nella GT e nella PRED)
        axes[i, 0].imshow(img, cmap='gray')
        axes[i, 0].set_title(f"ID: {stem}\nImmagine Originale")

        # Visualizzazione GT con etichette chiare
        axes[i, 1].imshow(m_gt, cmap='viridis', vmin=0, vmax=2)
        axes[i, 1].set_title(f"GT (Manuale)\nDanno Area: {np.sum(m_gt==2)} px")

        # Visualizzazione AI
        axes[i, 2].imshow(m_pred, cmap='viridis', vmin=0, vmax=2)
        axes[i, 2].set_title(f"PRED (UNet++)\nDanno Area: {np.sum(m_pred==2)} px")

        for ax in axes[i]: ax.axis('off')

    plt.tight_layout()
    plt.show()

visual_audit()

FINE CHECK CHE CAZZO

In [ ]:
# --- INSTALLAZIONI NECESSARIE ---
# scikit-learn, pandas, matplotlib e opencv sono di solito già su Colab
# installiamo seaborn per grafici più belli se serve aggiornarlo
#!pip install -q seaborn matplotlib scikit-learn
#@title cella che mi dice matrice confusione e R2 estraendo feautures dalle maschere
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, r2_score
from pathlib import Path
from tqdm.notebook import tqdm # Usa la versione notebook per barre di caricamento più carine
from google.colab import drive

## --- MONTAGGIO DRIVE (Se non lo hai già fatto) ---
#if not os.path.exists('/content/drive'):
#    drive.mount('/content/drive')

# ==============================================================================
# CONFIGURAZIONE (I TUOI PERCORSI)
# ==============================================================================

# Cartella con le 107 maschere "Golden" (Ground Truth manuali/corrette)
# Assicurati che i file dentro siano .png e abbiano valori 0, 1, 2
GT_DIR = Path("/content/drive/MyDrive/GOLDEN_GT_FIXED/masks")

# Cartella con tutte le 600 maschere predette dalla UNet
PRED_DIR = Path("/content/drive/MyDrive/UNETPPMaschereInferenza")

# Dove salvare i grafici risultanti
OUTPUT_PLOTS_DIR = Path("/content/drive/MyDrive/Tesi_Plots")
OUTPUT_PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# Fattore di scala (da Pixel a mm)
# Hai detto 512x512 pixel e foro da 6mm (che corrisponde a circa ~14mm di larghezza foto?)
# 35.8 px/mm corrisponde a un'immagine larga circa 14.3mm. È un valore plausibile per un crop 512.
SCALA_PX_MM = 35.8

# ==============================================================================
# LOGICA DI ELABORAZIONE
# ==============================================================================

def analizza_validazione():
    print("🚀 Inizio pipeline di validazione 'Tesi Legend'...")

    # 1. Recupero lista file e calcolo intersezione (match automatico)
    gt_files_map = {f.stem: f for f in GT_DIR.glob("*.png")}
    pred_files_map = {f.stem: f for f in PRED_DIR.glob("*.png")}

    # Intersezione basata sul nome del file (senza estensione)
    common_stems = sorted(list(set(gt_files_map.keys()) & set(pred_files_map.keys())))

    num_match = len(common_stems)
    print(f"📂 File Ground Truth trovati: {len(gt_files_map)}")
    print(f"📂 File Predizioni trovati: {len(pred_files_map)}")
    print(f"✅ Coppie valide (Intersezione): {num_match}")

    if num_match == 0:
        print("❌ ERRORE CRITICO: Nessuna corrispondenza trovata!")
        print("   Controlla che i nomi dei file nelle due cartelle siano identici.")
        print(f"   Esempio nome GT: {list(gt_files_map.keys())[0] if gt_files_map else 'Nessuno'}")
        print(f"   Esempio nome Pred: {list(pred_files_map.keys())[0] if pred_files_map else 'Nessuno'}")
        return

    # Liste per i dati
    # Usiamo liste standard di python per velocità nell'append, poi convertiamo in numpy
    # NOTA: Per la matrice di confusione su milioni di pixel, le liste potrebbero esplodere la RAM.
    # Ottimizzazione: Calcoliamo la matrice di confusione per ogni immagine e le sommiamo.

    total_conf_matrix = np.zeros((3, 3), dtype=np.int64) # Matrice accumulatore 3x3

    areas_gt = []
    areas_pred = []

    print("\n🔍 Elaborazione immagini...")

    for stem in tqdm(common_stems):
        # Carica i path
        path_gt = gt_files_map[stem]
        path_pred = pred_files_map[stem]

        # Lettura Maschere (Flag 0 per scala di grigi pura)
        mask_gt = cv2.imread(str(path_gt), 0)
        mask_pred = cv2.imread(str(path_pred), 0)

        # Check integrità
        if mask_gt is None or mask_pred is None:
            print(f"⚠️ Errore lettura file per {stem}, salto.")
            continue

        if mask_gt.shape != mask_pred.shape:
            # Resize predizione se dimensione diversa (non dovrebbe succedere se setup è corretto)
            mask_pred = cv2.resize(mask_pred, (mask_gt.shape[1], mask_gt.shape[0]), interpolation=cv2.INTER_NEAREST)

        # ---------------------------
        # A. AGGIORNAMENTO MATRICE DI CONFUSIONE
        # ---------------------------
        # Flattening rapido
        gt_flat = mask_gt.ravel()
        pred_flat = mask_pred.ravel()

        # Calcolo confusion matrix del singolo batch/immagine e somma al totale
        # labels=[0,1,2] assicura che la matrice sia sempre 3x3 anche se mancano classi in un'immagine
        current_cm = confusion_matrix(gt_flat, pred_flat, labels=[0, 1, 2])
        total_conf_matrix += current_cm

        # ---------------------------
        # B. CALCOLO METROLOGIA (AREE)
        # ---------------------------
        # Area danno (classe 2) in mm^2
        px_danno_gt = np.sum(mask_gt == 2)
        px_danno_pred = np.sum(mask_pred == 2)

        area_mm_gt = px_danno_gt / (SCALA_PX_MM**2)
        area_mm_pred = px_danno_pred / (SCALA_PX_MM**2)

        areas_gt.append(area_mm_gt)
        areas_pred.append(area_mm_pred)

    # ==============================================================================
    # GENERAZIONE OUTPUT E GRAFICI
    # ==============================================================================

    # 1. Plot Matrice di Confusione Normalizzata
    print("🎨 Generazione grafici...")

    # Normalizza per riga (somma su riga = 1) -> ci da Recall/Sensibilità per classe
    cm_norm = total_conf_matrix.astype('float') / total_conf_matrix.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(9, 7))
    labels = ['Sfondo', 'Foro', 'Danno']
    sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', cbar=True,
                xticklabels=labels, yticklabels=labels)
    plt.title(f'Analisi di Consistenza (Matrice Normalizzata)\nsu {num_match} campioni Golden Dataset', fontsize=14)
    plt.ylabel('Ground Truth (Reale)')
    plt.xlabel('Predizione UNet++')
    plt.tight_layout()
    plot_path_cm = OUTPUT_PLOTS_DIR / '01_Matrice_Confusione.png'
    plt.savefig(plot_path_cm, dpi=300)
    plt.show()
    print(f"💾 Grafico Matrice salvato in: {plot_path_cm}")

    # 2. Plot Regressione R^2 (Aree Danno)
    r2 = r2_score(areas_gt, areas_pred)

    plt.figure(figsize=(9, 9))

    # Scatter plot dei punti
    plt.scatter(areas_gt, areas_pred, alpha=0.6, s=40, color='#2c7bb6', edgecolors='k', linewidth=0.5, label='Campioni Validazione')

    # Linea ideale Y = X
    limit_max = max(max(areas_gt), max(areas_pred)) * 1.1
    plt.plot([0, limit_max], [0, limit_max], color='#d7191c', linestyle='--', linewidth=2, label='Fitting Ideale')

    plt.xlim(0, limit_max)
    plt.ylim(0, limit_max)
    plt.title(f'Capacità di Fitting Area Danno\n$R^2$ Score = {r2:.4f}', fontsize=14)
    plt.xlabel('Area Misurata Manualmente ', fontsize=12)
    plt.ylabel('Area Predetta dal Modello ', fontsize=12)
    plt.legend(loc='upper left', frameon=True)
    plt.grid(True, linestyle=':', alpha=0.6)

    plot_path_r2 = OUTPUT_PLOTS_DIR / '02_Regressione_Aree.png'
    plt.savefig(plot_path_r2, dpi=300)
    plt.show()
    print(f"💾 Grafico R^2 salvato in: {plot_path_r2}")

    print("\n✅ DONE. Sei pronto per la laurea.")

# Esegui tutto
analizza_validazione()

In [ ]:
#@title Cella per esportare i dati di area in CSV
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm # Usa la versione notebook per barre di caricamento più carine
import math

# --- Riusa le configurazioni definite in precedenza ---
# GT_DIR, PRED_DIR, SCALA_PX_MM dovrebbero essere già definiti dalla cella dgI31vlHz177
# Se non lo fossero, uncommenta e definiscile qui:
# GT_DIR = Path("/content/drive/MyDrive/GOLDEN_GT_FIXED/masks")
# PRED_DIR = Path("/content/drive/MyDrive/UNETPPMaschereInferenza")
# SCALA_PX_MM = 35.8

# Dove salvare i grafici e ora il CSV
OUTPUT_PLOTS_DIR = Path("/content/drive/MyDrive/Tesi_Plots")
OUTPUT_PLOTS_DIR.mkdir(parents=True, exist_ok=True)

def calculate_circularity(mask, class_val=2):
    """Calcola la circolarità per una specifica classe all'interno di una maschera."""
    # Assicurati che la maschera sia in formato CV_8UC1 per findContours
    binary_mask = (mask == class_val).astype(np.uint8) * 255

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0.0 # Nessun danno o contorno rilevato

    # Trova il contorno più grande, assumendo che sia il danno principale
    largest_contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(largest_contour)
    perimeter = cv2.arcLength(largest_contour, True)

    if perimeter == 0:
        return 0.0 # Evita la divisione per zero

    # Formula della circolarità (Shape factor)
    circularity = (4 * math.pi * area) / (perimeter * perimeter)
    return circularity

def export_area_data_to_csv():
    print("📊 Preparazione dati per l'esportazione CSV con metriche aggiuntive...")

    gt_files_map = {f.stem: f for f in GT_DIR.glob("*.png")}
    pred_files_map = {f.stem: f for f in PRED_DIR.glob("*.png")}
    common_stems = sorted(list(set(gt_files_map.keys()) & set(pred_files_map.keys())))

    if not common_stems:
        print("❌ Nessuna corrispondenza trovata per l'esportazione CSV.")
        return

    file_names = []
    px_areas_gt = []
    px_areas_pred = []
    areas_gt_mm2 = []
    areas_pred_mm2 = []
    circularity_gt = []
    circularity_pred = []

    print(f"🔍 Rielaborazione {len(common_stems)} immagini per dati di area e circolarità...")

    for stem in tqdm(common_stems):
        path_gt = gt_files_map[stem]
        path_pred = pred_files_map[stem]

        mask_gt = cv2.imread(str(path_gt), 0)
        mask_pred = cv2.imread(str(path_pred), 0)

        if mask_gt is None or mask_pred is None:
            print(f"⚠️ Errore lettura file per {stem}, salto.")
            continue

        if mask_gt.shape != mask_pred.shape:
            mask_pred = cv2.resize(mask_pred, (mask_gt.shape[1], mask_gt.shape[0]), interpolation=cv2.INTER_NEAREST)

        # --- Aree in Pixel ---
        px_danno_gt = np.sum(mask_gt == 2)
        px_danno_pred = np.sum(mask_pred == 2)
        px_areas_gt.append(px_danno_gt)
        px_areas_pred.append(px_danno_pred)

        # --- Aree in mm^2 ---
        area_mm_gt = px_danno_gt / (SCALA_PX_MM**2)
        area_mm_pred = px_danno_pred / (SCALA_PX_MM**2)
        areas_gt_mm2.append(area_mm_gt)
        areas_pred_mm2.append(area_mm_pred)

        # --- Circolarità ---
        circ_gt = calculate_circularity(mask_gt, class_val=2)
        circ_pred = calculate_circularity(mask_pred, class_val=2)
        circularity_gt.append(circ_gt)
        circularity_pred.append(circ_pred)

        file_names.append(stem)

    # Creazione DataFrame
    df_export = pd.DataFrame({
        'File_ID': file_names,
        'Pixel_Area_GT': px_areas_gt,
        'Pixel_Area_Predetta': px_areas_pred,
        'Area_GT_mm2': areas_gt_mm2,
        'Area_Predetta_mm2': areas_pred_mm2,
        'Circularity_GT': circularity_gt,
        'Circularity_Predetta': circularity_pred
    })

    # --- Calcolo della deviazione (errore) ---
    df_export['Error_Area_mm2'] = df_export['Area_Predetta_mm2'] - df_export['Area_GT_mm2']
    df_export['Error_Circularity'] = df_export['Circularity_Predetta'] - df_export['Circularity_GT']

    csv_path = OUTPUT_PLOTS_DIR / 'aree_danno_e_metrche_per_foro.csv'
    df_export.to_csv(csv_path, index=False)

    print(f"✅ Dati di area e metriche esportati con successo in: {csv_path}")
    print("Preview dei dati aggiornati:")
    print(df_export.head())

# Esegui la funzione di esportazione
export_area_data_to_csv()


In [ ]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
#ricreiamo dai file corretti le metriche di interesse
# --- CONFIGURAZIONE ---
GT_DIR = Path("/content/drive/MyDrive/GOLDEN_GT_FIXED/masks")
PRED_DIR = Path("/content/drive/MyDrive/UNETPPMaschereInferenza")

def detective_completo():
    gt_files = sorted(list(GT_DIR.glob("*.png")))
    report = []

    print(f"🕵️ Analisi di {len(gt_files)} file in corso...")

    for f in tqdm(gt_files):
        pred_f = PRED_DIR / f.name
        if not pred_f.exists():
            continue

        img_gt = cv2.imread(str(f), 0)
        img_pred = cv2.imread(str(pred_f), 0)

        # 1. Controllo Valori
        vals_gt = np.unique(img_gt)
        vals_pred = np.unique(img_pred)

        # 2. Calcolo Recall specifica per il DANNO (Classe 2) su questo file
        # Recall = (Pixel Danno beccati dall'AI) / (Totale Pixel Danno Reali)
        pixel_danno_reali = np.sum(img_gt == 2)
        pixel_danno_predetti_correttamente = np.sum((img_gt == 2) & (img_pred == 2))

        recall_danno = (pixel_danno_predetti_correttamente / pixel_danno_reali) if pixel_danno_reali > 0 else 1.0

        # 3. Controllo Inversione (Il Danno reale è finito nel Foro dell'AI?)
        danno_finito_nel_foro = np.sum((img_gt == 2) & (img_pred == 1))
        percentuale_inversione = (danno_finito_nel_foro / pixel_danno_reali) if pixel_danno_reali > 0 else 0

        report.append({
            'File': f.name,
            'GT_Vals': vals_gt,
            'PRED_Vals': vals_pred,
            'Recall_Danno': round(recall_danno * 100, 2),
            'Confusione_Danno_Foro': round(percentuale_inversione * 100, 2)
        })

    df_report = pd.DataFrame(report)

    # Ordiniamo per i peggiori risultati di Recall
    peggiori = df_report.sort_values(by='Recall_Danno').head(10)

    print("\n--- 📊 REPORT DIAGNOSTICO ---")
    print(f"Media Recall Danno: {df_report['Recall_Danno'].mean():.2f}%")
    print(f"Media Confusione Danno->Foro: {df_report['Confusione_Danno_Foro'].mean():.2f}%")

    print("\n--- 🚨 I 10 FILE PEGGIORI (Possibili inversioni o errori gravi) ---")
    print(peggiori[['File', 'Recall_Danno', 'Confusione_Danno_Foro']])

    return df_report

df_risultati = detective_completo()

In [ ]:
import cv2
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm

# --- CONFIGURAZIONE ---
# Cartella originale (quella con le inversioni)
GT_DIR_ORIGINAL = Path("/content/drive/MyDrive/GOLDEN_DATASET_CANONICO/Full_Images/masks")
# Nuova cartella su Drive per i file corretti
GT_DIR_FIXED = Path("/content/drive/MyDrive/GOLDEN_GT_FIXED/masks")
GT_DIR_FIXED.mkdir(parents=True, exist_ok=True)

def fix_ground_truth():
    files = sorted(list(GT_DIR_ORIGINAL.glob("*.png")))
    swapped_count = 0
    copied_count = 0

    print(f"🛠️ Inizio correzione su {len(files)} file...")

    for f in tqdm(files):
        img = cv2.imread(str(f), 0)
        if img is None: continue

        h, w = img.shape
        center_val = img[h//2, w//2] # Valore al centro (Foro)

        final_mask = img.copy()

        if center_val == 2:
            # INVERSIONE RILEVATA: Scambiamo 1 e 2
            # Usiamo un valore temporaneo (es. 99) per non sovrascrivere
            final_mask[img == 1] = 2
            final_mask[img == 2] = 1
            swapped_count += 1
        else:
            # File già corretto (centro = 1)
            copied_count += 1

        # Salva nella nuova cartella
        cv2.imwrite(str(GT_DIR_FIXED / f.name), final_mask)

    print(f"\n✅ OPERAZIONE COMPLETATA")
    print(f"🔄 File invertiti e corretti: {swapped_count}")
    print(f"📁 File mantenuti originali: {copied_count}")
    print(f"📍 I file corretti sono in: {GT_DIR_FIXED}")

fix_ground_truth()

In [ ]:
#@title Cella controllo visualizzazione immagini
#import cv2
#import numpy as np
import matplotlib.pyplot as plt


def visualizza_confronto(index=0):
    gt_files = sorted(list(GT_DIR.glob("*.png")))
    f = gt_files[index]
    pred_f = PRED_DIR / f.name

    gt = cv2.imread(str(f), 0)
    pred = cv2.imread(str(pred_f), 0)

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Mappa colori: 0=Nero, 1=Rosso (Foro), 2=Verde (Danno)
    ax[0].imshow(gt, cmap='nipy_spectral', vmin=0, vmax=2)
    ax[0].set_title(f"GROUND TRUTH (Manuale)\n{f.name}")

    ax[1].imshow(pred, cmap='nipy_spectral', vmin=0, vmax=2)
    ax[1].set_title(f"PREDIZIONE AI\nValori unici: {np.unique(pred)}")

    plt.show()

visualizza_confronto(72) # Cambia indice per vedere altri fori

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from scipy import stats

# ==============================================================================
# FASE 4: BOOTSTRAP STOCASTICO DELLE PRESTAZIONI (RECALL DANNO)
# ==============================================================================

def esegui_bootstrap_analysis(df_report, n_iterations=10000):
    """
    Analisi stocastica basata sui dati estratti dal detective_completo.
    Calcola l'intervallo di confidenza al 95% della Recall media.
    """
    # Estraiamo il vettore delle recall individuali (i tuoi 107 risultati)
    # Assumiamo che df_report abbia la colonna 'Recall_Danno'
    misure = df_report['Recall_Danno'].values / 100.0 # Torniamo in scala 0-1

    boot_means = []
    print(f"🎲 Avvio simulazione stocastica (Bootstrap {n_iterations} iterazioni)...")

    for i in range(n_iterations):
        # Campionamento con reinserimento (Stochastic Resampling)
        # Alcune immagini vengono contate doppie, altre saltate: simula nuovi dataset
        campione = resample(misure, replace=True, n_samples=len(misure))
        boot_means.append(np.mean(campione))

    boot_means = np.array(boot_means)

    # Calcolo Intervallo di Confidenza 95% (Percentile Method)
    alpha = 0.95
    p_lower = ((1.0 - alpha) / 2.0) * 100
    p_upper = (alpha + ((1.0 - alpha) / 2.0)) * 100
    ci_lower = np.percentile(boot_means, p_lower)
    ci_upper = np.percentile(boot_means, p_upper)

    # --- VISUALIZZAZIONE SCIENTIFICA ---
    plt.figure(figsize=(12, 6))

    # KDE plot: La "forma" della verità del tuo modello
    sns.histplot(boot_means, kde=True, color='royalblue', bins=50, stat="density", alpha=0.3)

    # Linee di controllo
    plt.axvline(np.mean(misure), color='red', linestyle='--', label=f'Media Osservata: {np.mean(misure):.3f}')
    plt.axvline(ci_lower, color='darkorange', linestyle=':', label=f'LCL (95%): {ci_lower:.3f}')
    plt.axvline(ci_upper, color='darkorange', linestyle=':', label=f'UCL (95%): {ci_upper:.3f}')

    # Coloriamo l'area di confidenza (High Value Content per tesi)
    x_fill = np.linspace(ci_lower, ci_upper, 100)
    plt.fill_between(x_fill, 0, stats.gaussian_kde(boot_means)(x_fill), color='orange', alpha=0.2)

    plt.title(f'Distribuzione Campionaria della Recall (Danno)\nBootstrap Stocastico (B={n_iterations})', fontsize=14)
    plt.xlabel('Valore Recall (Sensibilità)', fontsize=12)
    plt.ylabel('Densità di Probabilità', fontsize=12)
    plt.legend()
    plt.grid(alpha=0.2)

    plt.savefig(OUTPUT_PLOTS_DIR / 'bootstrap_recall_distribution.png', dpi=300)
    plt.show()

    print(f"\n--- 📈 RISULTATI BOOTSTRAP ---")
    print(f"Recall Media: {np.mean(misure):.4f}")
    print(f"Intervallo di Confidenza 95% (Stocastico): [{ci_lower:.4f}, {ci_upper:.4f}]")
    print(f"Margine d'errore della misura: ±{((ci_upper - ci_lower)/2):.4f}")

    return boot_means

# ESECUZIONE (Usa l'output della cella precedente 'df_risultati')
bootstrap_stats = esegui_bootstrap_analysis(df_risultati)